use newpycaret env

In [10]:
# packages
import snowflake.connector
import pandas as pd
import os
import numpy as np
from autots import AutoTS
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

2. Load Data
_______________________________________________________________

### To Do: copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username> <br>
%env snowflakepass=<your_snowflake_password>

In [4]:
# Query Snowflake

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('net_sales_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    df_in = pd.read_sql_query(query.read(),conn)

# Close the connection
conn.close()

C:\Users\crudek\AppData\Local\Temp\ipykernel_12204\3034601435.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_in = pd.read_sql_query(query.read(),conn)


In [5]:
### 1. Data prep ###

df = df_in

# convert month field to date
df["MONTH"] = pd.to_datetime(df["MONTH"])

# ***** ADJUSTABLE ***** #
# current params - want to get forecasts for 2023 (Jan - Dec) so holding out data 
df = df[df['MONTH'] <= pd.Timestamp(2022,12,1)]

# Some random months will have data that we want to remove (* Want to test without July though)
df = df[["DEP_ENT", "MONTH", "NET_SALES"]] # select fields of interest
df = df.sort_values(['DEP_ENT', 'MONTH']) # reorder dataframe


### 2. Spot checks ###

# check data types
print(df.dtypes)
print("-----")

# check the unique time_series. 14 different department-entity combinations
print(df['DEP_ENT'].nunique())
print("-----")

# check how many months for each dep-ent. 3 dep-ent do not have all months of data
print(df.groupby(['DEP_ENT']).size())
print("-----")

# check for nulls. No nulls
print(df.isnull().sum())

DEP_ENT              object
MONTH        datetime64[ns]
NET_SALES           float64
dtype: object
-----
14
-----
DEP_ENT
160_155    84
170_155    84
200_155    84
200_310    84
210_155    84
210_165    84
210_310    84
220_155    84
220_310    78
240_155    84
250_155    84
250_165    50
250_310    84
260_155    47
dtype: int64
-----
DEP_ENT      0
MONTH        0
NET_SALES    0
dtype: int64


# Multiple Series with auto TS
Tutorial: https://medium.com/data-science-in-your-pocket/automl-for-time-series-forecasting-using-autots-with-example-735fb559443a


https://winedarksea.github.io/AutoTS/build/html/source/tutorial.html

In [6]:
### Additional data prep ###

# create series
df_s = df.set_index(['DEP_ENT','MONTH'])['NET_SALES']
# convert back to dataframe
df_d = df_s.to_frame()
# impute
mean_imputer = SimpleImputer(strategy='mean')
df_d['NET_SALES'] = mean_imputer.fit_transform(df_d['NET_SALES'].values.reshape(-1,1))
#reset index
df_d.reset_index(inplace=True)

____________________________________________________________________________________________________________________________________________________________________________

In [7]:
### Time Series Loop ###

# Create empty dataframes
all_predictions = pd.DataFrame()

# list of each dep-ent
all_ts_ts = df_d['DEP_ENT'].unique()

for i in tqdm(all_ts_ts):
    
    # create subset
    df_subset = df_d[df_d['DEP_ENT'] == i]
    
    # trim
    df_subset_trim = df_subset[['MONTH', 'NET_SALES']]

    # model
    model = AutoTS(
        forecast_length=12,
        frequency='infer',
        prediction_interval=0.95,
        ensemble='simple',
        models_mode='deep',
        model_list = 'univariate', # or could do a list like ['ARIMA','ETS']
        max_generations=3,
        num_validations=3,
        no_negatives=True,
        n_jobs='auto')

    model = model.fit(
        df_subset_trim,
        date_col='MONTH', 
        value_col='NET_SALES',
    )
    
    print(model)

    # create prediction
    prediction = model.predict(forecast_length=12)
    
    # temp fcast dataframe
    temp_fcasts = prediction.forecast
    
    # rename
    temp_fcasts.rename(columns = {'index':'MONTH'}, inplace = True)
    
    temp_fcasts['DEP_ENT'] = i # add dep
    
    # append to master dataframe
    all_predictions = all_predictions.append(temp_fcasts)

  0%|          | 0/14 [00:00<?, ?it/s]

Using 6 cpus for n_jobs.
Data frequency is: MS, used frequency is: MS
Model Number: 1 with model ARIMA in generation 0 of 3
Model Number: 2 with model AverageValueNaive in generation 0 of 3
Model Number: 3 with model AverageValueNaive in generation 0 of 3
Model Number: 4 with model AverageValueNaive in generation 0 of 3
Model Number: 5 with model DatepartRegression in generation 0 of 3
Model Number: 6 with model DatepartRegression in generation 0 of 3
Model Number: 7 with model DatepartRegression in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 8 with model DatepartRegression in generation 0 of 3
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 8 in generation 0: DatepartRegression
Model Number: 9 with model ETS in generation 0 of 3
Model Number: 10 with model ETS in generation 0 of 3
Model Number: 11 with model GLM in generation 0 of 3
Model Number: 12 with model GLM in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:67: RuntimeWarning: invalid value encountered in divide
  (prior_mu / prior_sigma**2) + ((n * data_mu) / prior_sigma**2)

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:68: RuntimeWarning: divide by zero encountered in divide
  ) / ((1 / prior_sigma**2) + (n / prior_sigma**2))



ETS error ValueError('endog must be strictly positive when usingmultiplicative trend or seasonal components.')
ETS failed on NET_SALES with ValueError('endog must be strictly positive when usingmultiplicative trend or seasonal components.')
4 - Ensemble with avg smape 51.91: 
Model Number: 5 of 8 with model Ensemble for Validation 2
📈 5 - Ensemble with avg smape 50.16: 
Model Number: 6 of 8 with model Ensemble for Validation 2


WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:67: RuntimeWarning: divide by zero encountered in divide
  (prior_mu / prior_sigma**2) + ((n * data_mu) / prior_sigma**2)

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:68: RuntimeWarning: divide by zero encountered in divide
  ) / ((1 / prior_sigma**2) + (n / prior_sigma**2))

WARNING - (py.warnings._showwarnmsg) - c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\autots\tools\probabilistic.py:66: RuntimeWarning: invalid value encountered in divide
  post_mu = (



6 - Ensemble with avg smape 52.16: 
Model Number: 7 of 8 with model Ensemble for Validation 2
7 - Ensemble with avg smape 50.28: 
Model Number: 8 of 8 with model Ensemble for Validation 2
8 - Ensemble with avg smape 50.16: 
Initiated AutoTS object with best model: 
DatepartRegression
{'fillna': 'zero', 'transformations': {'0': 'MaxAbsScaler'}, 'transformation_params': {'0': {}}}
{'regression_model': {'model': 'ExtraTrees', 'model_params': {'n_estimators': 100, 'min_samples_leaf': 1, 'max_depth': None}}, 'datepart_method': 'recurring', 'polynomial_degree': None, 'regression_type': None}
Validation: 0, 1, 2
SMAPE: 33.50959284520726, 34.113227869593075, 49.72151214130358
MAE: 1490.446808333333, 1848.0827083333324, 2861.098341666669
SPL: 0.24797420210148904, 0.308908887930849, 1.4136967853206248


WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_12204\3078507374.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(temp_fcasts)

100%|██████████| 14/14 [3:00:03<00:00, 771.68s/it]


In [ ]:
# from autots.models.model_list import model_lists
# print(model_lists['univariate'])

# param_df = prediction.model_parameters
# param_df = pd.DataFrame.from_dict(param_df)

# upper_forecasts_df = prediction.upper_forecast
# lower_forecasts_df = prediction.lower_forecast

In [16]:
# rename index col
all_predictions.rename(columns={'index': 'MONTH', 'NET_SALES': 'PRED'}, inplace=True)

#initial sales pull
sales = df_in
# select cols
sales = sales[["DEP_ENT", "MONTH", "NET_SALES"]]
# only include sales data for the full months we have
first_of_month = datetime.today().replace(day=1).date()
sales = sales[sales['MONTH'] < pd.to_datetime(first_of_month)]

# combine prediction data and original sales data
merged = pd.merge(all_predictions, sales, how = 'left', on =['DEP_ENT', 'MONTH'])

In [18]:
### Budget ###

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('budget_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    budg = pd.read_sql_query(query.read(),conn)

# Close the connection
conn.close()

# convert month to datetime
budg["MONTH"] = pd.to_datetime(budg["MONTH"])
# select cols
budg = budg[["MONTH", "BUDGET_AMOUNT", "DEP_ENT"]]

WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_12204\3534014428.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  budg = pd.read_sql_query(query.read(),conn)



In [19]:
# combine prediction/sales data with budget data
merged2 = pd.merge(merged, budg, how = 'left', on =['DEP_ENT', 'MONTH'])

In [20]:
# write csv
merged2.to_csv('auto_ts_results.csv')